In [125]:
import pandas as pd
import requests
import json

# 1. Data Collection

### Collect from first data source ( Kaggle) and clean the dataset

The original dataset downloaded from Kaggle is in CSV format.

In [150]:
# Read the csv file
movies = pd.read_csv('../data/tmdb_movies_data.csv')

In [335]:
# The orginal dataset from Kaggle contains 10866 rows and 21 columns
movies.head()

,id,imdb_id,popularity,budget,revenue,original_title,cast,homepage,director,tagline,...,overview,runtime,genres,production_companies,release_date,vote_count,vote_average,release_year,budget_adj,revenue_adj
0,135397,tt0369610,32.985763,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,http://www.jurassicworld.com/,Colin Trevorrow,The park is open.,...,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,5562,6.5,2015,137999939.3,1.392446e+09
1,76341,tt1392190,28.419936,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,http://www.madmaxmovie.com/,George Miller,What a Lovely Day.,...,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,6185,7.1,2015,137999939.3,3.481613e+08
2,262500,tt2908446,13.112507,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,http://www.thedivergentseries.movie/#insurgent,Robert Schwentke,One Choice Can Destroy You,...,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,2480,6.3,2015,101199955.5,2.716190e+08
3,140607,tt2488496,11.173104,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,http://www.starwars.com/films/star-wars-episod...,J.J. Abrams,Every generation has a story.,...,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,5292,7.5,2015,183999919.0,1.902723e+09
4,168259,tt2820852,9.335014,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,http://www.furious7.com/,James Wan,Vengeance Hits Home,...,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,2947,7.3,2015,174799923.1,1.385749e+09


In [329]:
# Select imdb_id and budget of movies released between 2012 and 2015 with nonzero budget
# Drop the rows with missing values
# Save the data as a dataframe

budget = movies.loc[(movies['release_year'] > 2011) & (movies['release_year'] < 2016) & (movies['budget'] > 0), ['imdb_id','budget']]
budget = budget.dropna().reset_index(drop=True)

In [330]:
# the dataframe has 953 rows and 2 columns
budget

,imdb_id,budget
0,tt0369610,150000000
1,tt1392190,150000000
2,tt2908446,110000000
3,tt2488496,200000000
4,tt2820852,190000000
...,...,...
948,tt2319018,1400000
949,tt2187884,650
950,tt2403029,4500000
951,tt1966566,30000000


### Collect from second data source (RapidAPI) and clean the dataset

In [307]:
# Collect metascores from API 
url = "https://movie-database-alternative.p.rapidapi.com/"

metascore = []

for tt_id in budget['imdb_id']:
    querystring = {"r":"json","i":tt_id}

    headers = {
        "X-RapidAPI-Key": "566b74018dmsh42a245d474fa5f9p18dd45jsnf6ba0fd74133",
        "X-RapidAPI-Host": "movie-database-alternative.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    # get metascore by key ['Metascore']
    score = response.json()['Metascore']
    metascore.append(float(score if score != 'N/A' else 'NaN'))

In [312]:
# add attribute Metascore to the previous budget dataframe
budget['Metascore']=metascore

In [313]:
# the dataframe has 953 rows and 3 columns now
budget

,imdb_id,budget,Metascore
0,tt0369610,150000000,59.0
1,tt1392190,150000000,90.0
2,tt2908446,110000000,42.0
3,tt2488496,200000000,80.0
4,tt2820852,190000000,67.0
...,...,...,...
948,tt2319018,1400000,NaN
949,tt2187884,650,58.0
950,tt2403029,4500000,NaN
951,tt1966566,30000000,49.0


In [316]:
# drop the rows with missing value of metascore
budget_metascore = budget.dropna().reset_index(drop=True)

In [323]:
# the clean dataframe has 777 rows and 3 columns
budget_metascore

,imdb_id,budget,Metascore
0,tt0369610,150000000,59.0
1,tt1392190,150000000,90.0
2,tt2908446,110000000,42.0
3,tt2488496,200000000,80.0
4,tt2820852,190000000,67.0
...,...,...,...
772,tt2361700,1000000,69.0
773,tt1509788,3000000,37.0
774,tt2234155,58000000,42.0
775,tt2187884,650,58.0


In [317]:
# save the dataframe as a csv file: budget_metascore.csv
budget_metascore.to_csv('../data/budget_metascore.csv')

# 2. Analysis and Visualization